In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import pandas as pd
import numpy as np
import scanpy as sc

/ihome/djishnu/alw399/.local/lib/python3.9/site-packages/louvain/__init__.py:54: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import get_distribution, DistributionNotFound


In [4]:
# adata = sc.read_h5ad('/ix3/djishnu/alw399/SLIDE_PLM/data/alyx/tcells/seacells_adata.h5ad')
# adata

In [ ]:
x = pd.read_csv('example_data/HIV+cytokines_X.csv', index_col=0)
y = pd.read_csv('example_data/HIV+cytokines_y.csv', index_col=0)

In [9]:
x.head(3)

,4+GT2+_CF,4+GT2+_ES,4+GT2+_PP,4+GT2+_TB,4+Gt2-_CF,4+Gt2-_ES,4+Gt2-_PP,4+Gt2-_TB,4+GT2-_CF,4+GT2-_ES,...,8+G+_PP,8+G+_TB,8+2+_CF,8+2+_ES,8+2+_PP,8+2+_TB,8+T+_CF,8+T+_ES,8+T+_PP,8+T+_TB
PATIENT ID,,,,,,,,,,,,,,,,,,,,,
TB-1013,-0.351467,-0.494833,0.020765,-0.298490,-0.312934,-0.324441,-0.549784,-0.312218,-0.266078,-0.226816,...,-0.354071,1.044139,-0.577204,-0.512279,-0.403687,-0.350745,-0.315942,-0.246482,-0.086242,0.103430
TB-1017,-0.319115,-0.349612,-0.251067,-0.346845,-0.312934,-0.324441,-0.156850,-0.312218,-0.237711,-0.170521,...,-0.344015,-0.425125,1.207447,-0.512279,0.009329,-0.113176,1.163932,-0.233130,0.097271,-0.214786
TB-1024,-0.288968,2.705440,0.204968,-0.212607,-0.312934,-0.324441,0.458331,-0.312218,-0.116112,0.482073,...,-0.454626,-0.514171,0.350815,0.090282,-0.208048,-0.469529,-0.359625,0.252891,-0.485610,-0.360257


In [10]:
y.head(3)

,Class_Class
PATIENT ID,
TB-1013,1
TB-1017,1
TB-1024,1


In [6]:
input_params = {
    'x_path' : None,
    'y_path' : None,
    'fdr' : 0.1,
    'thresh_fdr': 0.1,
    'spec' : 0.1,
    'y_factor': True,
    'niter' : 500,
    'SLIDE_top_feats': 20,
    'pure_homo' : True,
    'delta' : [0.01, 0.1],
    'lambda' : [0.1, 0.5],
    'out_path': '/ix3/djishnu/alw399/SLIDE_PLM/data/slide/outputs/alyx/bcells'
}

In [12]:
import sys
sys.path.append('src/SLIDE')

In [13]:
from slide import OptimizeSLIDE

slider = OptimizeSLIDE(
    input_params, x=x, y=y)

In [ ]:
%%time
slider.run_pipeline(verbose=True, n_workers=1)

In [7]:
# slider.create_summary_table(slider.input_params['out_path'])


### Debugging

In [ ]:
out_iter = '/ix3/djishnu/alw399/SLIDE_py/example_results/slide_py/0.1_0.5_out/love_result.pkl'

slider.show_params()
slider.load_love(out_iter)

In [ ]:
n_workers = 1
verbose=True

slider.run_SLIDE(
    latent_factors=slider.latent_factors, 
    niter=slider.input_params['niter'], 
    spec=slider.input_params['spec'], 
    fdr=slider.input_params['fdr'],
    verbose=verbose,
    n_workers=n_workers,
    outpath=out_iter
)

In [ ]:
from knockoffs import Knockoffs

machop = Knockoffs(slider.data['Y'].values, slider.latent_factors.values)
machop 

In [ ]:
result = machop.filter_knockoffs_iterative(machop.z2, machop.y, fdr=0.1, niter=10)
result

In [7]:
delta = 0.1
lbd = 0.5

out_iter = input_params['out_path'] + f'/{delta}_{lbd}_out'

In [8]:
slider.load_love(out_iter + '/love_result.pkl')

In [67]:
slider.sig_LFs = np.loadtxt(out_iter + '/sig_LFs.txt', dtype=str).reshape(-1).tolist()
slider.sig_interacts = np.loadtxt(out_iter + '/sig_interacts.txt', dtype=str).reshape(-1).tolist()

In [ ]:
from knockoffs import Knockoffs

n = slider.latent_factors.shape[0]

slider.interaction_terms = Knockoffs.get_interaction_terms(
    slider.latent_factors[slider.sig_LFs].values, 
    slider.latent_factors[slider.sig_interacts].values
).reshape(n, -1)

slider.interaction_terms.shape

In [ ]:
from plotting import Plotter

sig_LF_genes = {str(lf): slider.get_LF_genes(
    lf=lf, 
    top_feats=20,
    outpath=out_iter) for lf in slider.sig_LFs}
_ = Plotter.plot_latent_factors(sig_LF_genes, loadings=slider.A, outdir=out_iter, title='marginal_LFs')

In [ ]:
sig_interact_genes = {str(lf): slider.get_LF_genes(
    lf=lf, 
    top_feats=20,
    outpath=out_iter) for lf in slider.sig_interacts}
_ = Plotter.plot_latent_factors(sig_interact_genes, loadings=slider.A, outdir=out_iter, title='interaction_LFs')